In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import pyplot as plt
import seaborn as sns

/Users/tajo5912/jhub_venv/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
workdir = '/scratch/Users/tajo5912/peptide/ChIP-seq/counts/'
expt = pd.read_csv(workdir + 'counts_over_TSS_1000bp.txt', sep = '\t', header=None)
shuff= pd.read_csv(workdir + 'shuffle_counts_over_TSS_1000bp.txt', sep='\t', header=None)

# normalization

In [62]:
count_names = ['DMSO_R1','DMSO_R2', 'DMSO_pep_R1','DMSO_pep_R2',
        'Nutlin_R1','Nutlin_R2','Nutlin_pep_R1','Nutlin_pep_R2',
        'in_DMSO_R1','in_DMSO_R2','in_DMSO_pep_R1','in_DMSO_pep_R2',
        'in_Nutlin_R1','in_Nutlin_R2','in_Nutlin_pep_R1','in_Nutlin_pep_R2']

In [4]:
cols = ['chr', 'start', 'stop', 'accession', 'score', 'strand', '1', '2', '3', '4', '5', '6']
cols.extend(count_names)
expt.columns = cols
cols = ['chr', 'start', 'stop', 'accession']
cols.extend(count_names)
expt = expt[cols]

scols = ['chr', 'start','stop']
scols.extend(count_names)
shuff.columns = scols

expt['len'] = expt['stop'] - expt['start']
shuff['len'] = shuff['stop'] - shuff['start']

olenexpt = len(expt)
print(olenexpt)

expt['sample_counts'] = expt[count_names[0:8]].sum(axis=1)
expt = expt[(expt['sample_counts'] > 0)]

slenexpt = len(expt)
print(slenexpt)
print('dropped: ' + str(olenexpt-slenexpt))
olenshuff = len(shuff)
print(olenshuff)

shuff['total_counts'] = shuff[count_names].sum(axis=1)
shuff = shuff[(shuff['total_counts'] > 0)]

tlenshuff = len(shuff)
print(tlenshuff)
print('dropped: ' + str(olenshuff-tlenshuff))

expt_cut = expt.accession.str.split("_", expand=True,)
expt_cut = expt_cut[[2]]
expt_cut.columns = ['gene']
expt = expt.merge(expt_cut,  left_index=True, right_index=True)

expt = expt.sort_values(by='sample_counts', ascending=False)
expt= expt.drop_duplicates(subset=['gene'], keep='first')
expt

82500
71996
dropped: 10504
82500
77370
dropped: 5130


,chr,start,stop,accession,DMSO_R1,DMSO_R2,DMSO_pep_R1,DMSO_pep_R2,Nutlin_R1,Nutlin_R2,...,in_DMSO_R2,in_DMSO_pep_R1,in_DMSO_pep_R2,in_Nutlin_R1,in_Nutlin_R2,in_Nutlin_pep_R1,in_Nutlin_pep_R2,len,sample_counts,gene
696,chr1,16740215,16741215,NR_004430_RNU1-1,5033,2243,0,2228,3284,1860,...,38,99,65,49,39,51,60,1000,21637,RNU1-1
695,chr1,16740215,16741215,NR_004427_RNU1-2,5033,2243,0,2228,3284,1860,...,38,99,65,49,39,51,60,1000,21637,RNU1-2
694,chr1,16740215,16741215,NR_004421_RNU1-4,5033,2243,0,2228,3284,1860,...,38,99,65,49,39,51,60,1000,21637,RNU1-4
693,chr1,16740215,16741215,NR_004408_RNU1-3,5033,2243,0,2228,3284,1860,...,38,99,65,49,39,51,60,1000,21637,RNU1-3
692,chr1,16740215,16741215,NR_004400_RNVU1-18,5033,2243,0,2228,3284,1860,...,38,99,65,49,39,51,60,1000,21637,RNVU1-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64109,chr6,63319677,63320677,NM_001143940_LGSN,0,0,0,0,0,1,...,0,0,0,0,0,0,1,1000,1,LGSN
19493,chr13,86911619,86912619,NR_132371_LINC00430,0,0,0,0,1,0,...,0,0,1,0,0,0,1,1000,1,LINC00430
64125,chr6,68055050,68056050,NR_125869_LOC102723883,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1000,1,LOC102723883
56933,chr4,60922318,60923318,NR_039652_MIR548AG1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1000,1,MIR548AG1


In [2]:
##pull in millions mapped
mm= pd.read_csv('/scratch/Users/tajo5912/peptide/ChIP-seq/qc/chip_millions_mapped.txt', sep='\t')
mm=mm[['sample', 'millionsmapped']]
mm = mm.sort_values(by='sample')
mm= mm.reset_index(drop=True)
mm['labs'] = count_names

##pull in p53 targets
p53 = pd.read_csv('/Users/tajo5912/temp/p53_target_andrysik2018.txt', sep='\t', header=None)
p53.columns = ['gene', 'pathway']

In [8]:
#filter out all significant genes and p53 genes
deseq_dir1 = '/scratch/Users/tajo5912/peptide/RNA-seq/deseq2/expt1/'
deseq_dir2 = '/scratch/Users/tajo5912/peptide/RNA-seq/deseq2/expt2_limma_correction/'
comparisons = ['DPvNP', 'DPvNW', 'DWvDP', 'DWvNP', 'DWvNW', 'NWvNP']
gl=[]
for comparison in comparisons:
    print(comparison)
    sq1 = pd.read_csv(deseq_dir1 + comparison +'/diffexpr-results_DESeq2.txt', sep='\t')
    sq1 = sq1[(sq1['padj'] < 0.05)]
    g1 = list(sq1['Gene'])
    
    sq2 = pd.read_csv(deseq_dir2 + comparison +'/diffexpr-results_DESeq2.txt', sep='\t')
    sq2 = sq2[(sq2['padj'] < 0.05)]
    g2 = list(sq2['Gene'])
    
    gl.extend(g2)
    gl.extend(g1)

gl.extend(list(p53['gene']))
print(len(gl))
gs=(set(gl))
gl=list(gs)
print(len(gl))

egl = set(expt['gene'])
len(egl.difference(set(gl)))
gg = list(egl.difference(set(gl)))
len(gg)
expt=expt[(expt['gene'].isin(gg))]

DPvNP
DPvNW
DWvDP
DWvNP
DWvNW
NWvNP
3135
2820


In [11]:
##getting ratios
def no0s(row):
    if (row['in_' + i] == 0):
        return row['in_' + i] + 0.99999
    else:
        return row['in_' + i]

dd={}
lsums_counts=[]
lsums_in_norm_counts=[]
n_cuts=[]
for i in count_names[0:8]:
    mini_expt = expt[['chr', 'start', 'stop', 'accession', i, 'in_' + i, 'len']]
    mini_expt['in_' + i] = mini_expt.apply(lambda row: no0s(row), axis=1)
    mini_expt['in_normalized_' + i] = (mini_expt[i])/(mini_expt['in_'+i])
    n_counts = len(mini_expt)
    mini_expt = mini_expt[(mini_expt['in_normalized_'+i] > 0)]
    a_0 = len(mini_expt)
    q=0.25

    quant = mini_expt['in_normalized_' + i].quantile(q)
    print('Initially there are ' + str(n_counts) + ' counts in ' + i)

    mini_expt = mini_expt[(mini_expt['in_normalized_'+i] >= quant)]
    n_cut= len(mini_expt)
    print('Now there are ' + str(n_cut) + ' counts remaining.')
    print('This is ' + str(n_cut/n_counts*100) + ' percent of the original counts for ' + i)

    dd[i] = mini_expt
    lsums_counts.append(sum(mini_expt[i]))
    lsums_in_norm_counts.append(sum(mini_expt['in_normalized_' + i]))
    n_cuts.append(n_cut)


Initially there are 25345 counts in DMSO_R1
Now there are 18580 counts remaining.
This is 73.30834484119156 percent of the original counts for DMSO_R1
Initially there are 25345 counts in DMSO_R2
Now there are 17873 counts remaining.
This is 70.5188400078911 percent of the original counts for DMSO_R2
Initially there are 25345 counts in DMSO_pep_R1
Now there are 6 counts remaining.
This is 0.02367330834484119 percent of the original counts for DMSO_pep_R1
Initially there are 25345 counts in DMSO_pep_R2
Now there are 18040 counts remaining.
This is 71.17774709015585 percent of the original counts for DMSO_pep_R2
Initially there are 25345 counts in Nutlin_R1
Now there are 19000 counts remaining.
This is 74.96547642533044 percent of the original counts for Nutlin_R1
Initially there are 25345 counts in Nutlin_R2
Now there are 18381 counts remaining.
This is 72.523180114421 percent of the original counts for Nutlin_R2
Initially there are 25345 counts in Nutlin_pep_R1
Now there are 13255 count

In [12]:
ss={}
shuff_lsums_counts=[]
shuff_n_cuts=[]
for i in count_names[0:8]:
    mini_shuff = shuff[['chr', 'start', 'stop', i, 'len']]
    n_counts = len(mini_shuff)
    lq=0.1
    hq = 0.9
    low_quant= mini_shuff[i].quantile(lq)
    high_quant = mini_shuff[i].quantile(hq)
    
    mini_shuff = mini_shuff[(mini_shuff[i] > low_quant) & 
                                (mini_shuff[i] < high_quant)]
    n_cut= len(mini_shuff)
    print('Now there are ' + str(n_cut) + ' counts remaining')
    print('This is ' + str(n_cut/n_counts*100) + ' percent of the original counts.')

    ss[i] = mini_shuff
    shuff_lsums_counts.append(sum(mini_shuff[i]))
    shuff_n_cuts.append(n_cut)

Now there are 57337 counts remaining
This is 74.10753522036964 percent of the original counts.
Now there are 48857 counts remaining
This is 63.14721468269355 percent of the original counts.
Now there are 0 counts remaining
This is 0.0 percent of the original counts.
Now there are 50705 counts remaining
This is 65.5357373659041 percent of the original counts.
Now there are 57933 counts remaining
This is 74.87785963551764 percent of the original counts.
Now there are 54792 counts remaining
This is 70.81814656843738 percent of the original counts.
Now there are 14897 counts remaining
This is 19.254232906811424 percent of the original counts.
Now there are 49166 counts remaining
This is 63.54659428719142 percent of the original counts.


In [13]:
ratios = [lsums_counts, lsums_in_norm_counts, n_cuts, shuff_lsums_counts, shuff_n_cuts]
df = pd.DataFrame(ratios, columns = [count_names[0:8]])
df = df.transpose().reset_index()
df.columns = ['sample' , 'tss_sum_counts', 'tss_sum_input_norm_counts', 'tss_n_cuts',
                          'shuff_sum_counts', 'shuff_n_cuts']
df['tss_av'] = df['tss_sum_counts']/df['tss_n_cuts']
df['shuff_av'] = df['shuff_sum_counts']/df['shuff_n_cuts']
df['snr'] = df['tss_av']/df['shuff_av']/10
df

,sample,tss_sum_counts,tss_sum_input_norm_counts,tss_n_cuts,shuff_sum_counts,shuff_n_cuts,tss_av,shuff_av,snr
0,DMSO_R1,1635572.0,315178.478115,18580.0,333894.0,57337.0,88.028633,5.823360,1.511647
1,DMSO_R2,1465304.0,203787.990536,17873.0,238366.0,48857.0,81.984222,4.878851,1.680400
2,DMSO_pep_R1,6.0,1.592857,6.0,0.0,0.0,1.000000,NaN,NaN
3,DMSO_pep_R2,1322097.0,104190.509914,18040.0,223818.0,50705.0,73.286973,4.414121,1.660285
4,Nutlin_R1,933027.0,149941.818860,19000.0,517857.0,57933.0,49.106684,8.938895,0.549360
5,Nutlin_R2,1224435.0,228473.161728,18381.0,273017.0,54792.0,66.614167,4.982789,1.336885
6,Nutlin_pep_R1,2557102.0,596055.297735,13255.0,201082.0,14897.0,192.916032,13.498154,1.429203
7,Nutlin_pep_R2,1688326.0,177197.544578,17966.0,294656.0,49166.0,93.973394,5.993085,1.568030


In [ ]:
mm.columns = ['file_name', 'millions_mapped', 'sample']
df = df.merge(mm, on='sample')
df.to_csv('/scratch/Users/tajo5912/peptide/ChIP-seq/annotations/ratio_adj.txt', index=False, sep='\t')

In [8]:
ratio_app=pd.read_csv('/scratch/Users/tajo5912/peptide/ChIP-seq/annotations/ratio_adj.txt', sep='\t')
ratio_app = ratio_app[['sample','file_name_x','snr']]
ratio_app = ratio_app.fillna(0.1)
ratio_app

,sample,file_name_x,snr
0,DMSO_R1,DMSO_R1_NEW_S3_R1_001,1.511647
1,DMSO_R2,DMSO_R2_NEW_S4_R1_001,1.680400
2,DMSO_pep_R1,DMSO_pep_R1_S1_R1_001,0.100000
3,DMSO_pep_R2,DMSO_pep_R2_S2_R1_001,1.660285
4,Nutlin_R1,Nutlin_R1_NEW_S15_R1_001,0.549360
5,Nutlin_R2,Nutlin_R2_NEW_S17_R1_001,1.336885
6,Nutlin_pep_R1,Nutlin_pep_R1_S13_R1_001,1.429203
7,Nutlin_pep_R2,Nutlin_pep_R2_S14_R1_001,1.568030


In [9]:
convert_dict = {}
convert_dict_app_rat = {}
for i in count_names[0:8]:
    convert_dict[i] = int
    expt = expt.astype(convert_dict)
    vcut = ratio_app[ratio_app['sample'] == i]
    v = float(vcut['snr'])
    expt[i+'_applied_ratio'] = expt[i]/v
    convert_dict_app_rat[i + '_applied_ratio'] = float
    expt = expt.astype(convert_dict_app_rat)


In [15]:
expt.columns
# expt.merge(p53, on='gene')
exptout=expt[['chr', 'start', 'stop', 'accession', 'DMSO_R1', 'DMSO_R2',
       'DMSO_pep_R2', 'Nutlin_R1', 'Nutlin_R2',
       'Nutlin_pep_R2', 'in_DMSO_R1', 'in_DMSO_R2', 'in_DMSO_pep_R1',
       'in_DMSO_pep_R2', 'in_Nutlin_R1', 'in_Nutlin_R2', 'in_Nutlin_pep_R1',
       'in_Nutlin_pep_R2',
       'DMSO_R1_applied_ratio', 'DMSO_R2_applied_ratio',
       'DMSO_pep_R2_applied_ratio',
       'Nutlin_R1_applied_ratio', 'Nutlin_R2_applied_ratio',
       'Nutlin_pep_R2_applied_ratio']]
exptout.to_csv('/scratch/Users/tajo5912/peptide/ChIP-seq/counts/tss_adjusted_counts.txt', index=False, sep='\t')

# bedgraph generation

In [85]:
samps=range(8)
# ratio_app.loc[samp]
for samp in samps:
    print(samp)
    print(ratio_app.loc[samp])

0
sample                       DMSO_R1
file_name_x    DMSO_R1_NEW_S3_R1_001
snr                          1.51165
Name: 0, dtype: object
1
sample                       DMSO_R2
file_name_x    DMSO_R2_NEW_S4_R1_001
snr                           1.6804
Name: 1, dtype: object
2
sample                   DMSO_pep_R1
file_name_x    DMSO_pep_R1_S1_R1_001
snr                              0.1
Name: 2, dtype: object
3
sample                   DMSO_pep_R2
file_name_x    DMSO_pep_R2_S2_R1_001
snr                          1.66028
Name: 3, dtype: object
4
sample                        Nutlin_R1
file_name_x    Nutlin_R1_NEW_S15_R1_001
snr                             0.54936
Name: 4, dtype: object
5
sample                        Nutlin_R2
file_name_x    Nutlin_R2_NEW_S17_R1_001
snr                             1.33689
Name: 5, dtype: object
6
sample                    Nutlin_pep_R1
file_name_x    Nutlin_pep_R1_S13_R1_001
snr                              1.4292
Name: 6, dtype: object
7
sample             

In [ ]:
for samp in samps:
    print('Reading in: ' + ratio_app['file_name'].loc[samp])
    print('SNR Value:')
    print(float(ratio_app['snr'].loc[samp]))
    bg = pd.read_csv('/scratch/Users/tajo5912/peptide/ChIP-seq/mapped/bedgraphs/' + ratio_app['file_name'].loc[samp] + '.bedGraph', 
                     sep='\t', header=None)
    bg.columns = ['chr', 'start', 'stop', 'level']
    bg['level_adj'] = bg['level']/float(ratio_app['snr'].loc[samp])
    bg = bg[['chr', 'start', 'stop', 'level_adj']]
    outdir = '/scratch/Users/tajo5912/peptide/ChIP-seq/mapped/chipin_adjusted/bedgraphs/'
    bg.to_csv(outdir + ratio_app['sample'].loc[samp] + '.bedGraph', sep='\t', header=None, index=False)

# bigwig generation

In [ ]:
bedgraph_dir= '/scratch/Users/tajo5912/peptide/ChIP-seq/mapped/chipin_adjusted/bedgraphs/'
bigwig_dir = '/scratch/Users/tajo5912/peptide/ChIP-seq/mapped/chipin_adjusted/bigwigs/'
chrom_sizes = '/scratch/Users/tajo5912/peptide/ChIP-seq/mapped/ratio_adjusted/hg38.chrom.sizes'

In [ ]:
samps=range(8)
for samp in samps:
    print('Converting: ' + str(ratio_app['sample'].loc[samp]))
    os.system('~/bin/x86_64/./bedGraphToBigWig ' + 
           bedgraph_dir + str(ratio_app['sample'].loc[samp]) + '.bedGraph ' 
           + chrom_sizes + ' ' + 
           bigwig_dir + str(ratio_app['sample'].loc[samp]) + '.bw')

# chip heat

In [3]:
df = pd.read_csv('/Users/tajo5912/peptide/tss_adjusted_counts.txt', sep='\t')
df_cut = df.accession.str.split("_", expand=True,)
df_cut = df_cut[[2]]
df_cut.columns = ['gene']
df = df.merge(df_cut,  left_index=True, right_index=True)

In [ ]:
dp53 = df.merge(p53, on ='gene')
av = ['DMSO','Nutlin']
for i in av:
    dp53[i + '_ar'] = (dp53[i + '_R1_applied_ratio'] + dp53[i + '_R1_applied_ratio'])/2

In [104]:
dp53['Nutlin_ar']=dp53['Nutlin_ar'].replace(0,1)
dp53['Nutlin_pep_R2_applied_ratio']=dp53['Nutlin_pep_R2_applied_ratio'].replace(0,1)

dp53['np/n'] = dp53['Nutlin_pep_R2_applied_ratio']/dp53['Nutlin_ar']
dp53['log2_np/n'] = np.log2(dp53['np/n'])
dp53.describe()
dp53=dp53.sort_values(by='log2_np/n', ascending=False)
dp53=dp53[(dp53['gene'].isin(p53_genes))]

,chr,start,stop,accession,DMSO_R1,DMSO_R2,DMSO_pep_R1,DMSO_pep_R2,Nutlin_R1,Nutlin_R2,...,Nutlin_R1_applied_ratio,Nutlin_R2_applied_ratio,Nutlin_pep_R1_applied_ratio,Nutlin_pep_R2_applied_ratio,t,DMSO_ar,Nutlin_ar,Nutlin_pep_ar,np/n,log2_np/n
31975,chr17,74859585,74860585,NM_000835_GRIN2C,38,50,0,65,60,135,...,109.218064,100.981009,157.430392,146.680822,1.56803,25.138149,109.218064,157.430392,1.343009,0.425469
63506,chr6,36678413,36679413,NM_000389_CDKN1A,221,243,0,205,326,595,...,593.418145,445.064446,960.675236,658.788213,1.56803,146.198180,593.418145,960.675236,1.110159,0.150766
14308,chr11,125569176,125570176,NR_110770_EI24,178,146,0,133,84,169,...,152.905289,126.413263,281.275634,161.986647,1.56803,117.752380,152.905289,281.275634,1.059392,0.083237
59345,chr5,58459786,58460786,NM_006622_PLK2,312,333,0,357,188,355,...,342.216599,265.542653,417.015616,362.237856,1.56803,206.397430,342.216599,417.015616,1.058505,0.082028
738,chr1,18480629,18481629,NM_152375_KLHDC7A,13,12,0,20,14,22,...,25.484215,16.456164,27.987625,26.147451,1.56803,8.599893,25.484215,27.987625,1.026025,0.037066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64674,chr6,106086035,106087035,NM_001198_PRDM1,29,20,0,18,18,20,...,32.765419,14.960149,51.777107,13.392597,1.56803,19.184377,32.765419,51.777107,0.408742,-1.290738
17674,chr12,108561162,108562162,NM_014301_ISCU,71,27,0,33,40,27,...,72.812042,20.196202,16.092885,28.698422,1.56803,46.968646,72.812042,16.092885,0.394144,-1.343206
59647,chr5,73565339,73566339,NM_023039_ANKRA2,114,88,0,61,104,66,...,189.311310,49.368493,71.368444,70.151697,1.56803,75.414446,189.311310,71.368444,0.370563,-1.432211
54796,chr3,151316520,151317520,NM_023915_GPR87,3,2,0,4,14,5,...,25.484215,3.740037,0.000000,1.913228,1.56803,1.984591,25.484215,1.000000,0.075075,-3.735523


In [105]:
viridis = cm.get_cmap('viridis', 256)
newcolors = viridis(np.linspace(0, 1, 256))
pink = np.array([248/256, 24/256, 148/256, 1])
newcolors[:25, :] = pink
newcmp = ListedColormap(newcolors)


def plot_examples(cms):
    """
    helper function to plot two colormaps
    """
    np.random.seed(19680801)
    data = np.random.randn(30, 30)

    fig, axs = plt.subplots(1, 2, figsize=(6, 3), constrained_layout=True)
    for [ax, cmap] in zip(axs, cms):
        psm = ax.pcolormesh(data, cmap=cmap, rasterized=True, vmin=-4, vmax=4)
        fig.colorbar(psm, ax=ax)
    plt.show()

#plot_examples([viridis, newcmp])


top = cm.get_cmap('Blues_r', 128)
bottom = cm.get_cmap('YlOrBr', 128)

newcolors = np.vstack((top(np.linspace(0, 1, 128)),
                       bottom(np.linspace(0, 1, 128))))
newcmp = ListedColormap(newcolors, name='OrangeBlue')
#plot_examples([viridis, newcmp])

In [106]:
plt = sns.clustermap(dp53[['log2_np/n']], cmap=newcmp, 
                     yticklabels=dp53['gene'], method='ward', figsize=(8, 16), 
                     row_cluster=False, col_cluster=False, vmin=-4.7, vmax=4.7)

plt.savefig(workdir+'R12Nut_R2Nutpep_logFC_chip_p53.pdf')

In [ ]:
applied_ratio_names = []
for i in count_names[0:8]:
    applied_ratio_names.append(i+'_applied_ratio')
dp53['sum_applied_ratio'] = dp53[applied_ratio_names].sum(axis=1)

plt = sns.clustermap(dp53[['sum_applied_ratio']], cmap='Greys', 
                     yticklabels=dp53['gene'], method='ward', figsize=(8, 16), 
                     row_cluster=False, col_cluster=False, vmin=0, vmax=3100)

plt.savefig(workdir+'read_density_chip_p53.pdf')